In [2]:
import json
import requests
from pprint import pprint

In [3]:
print("Based on the confirmed details, here is the Master Scenario Events List: \n\n1. 00:00 - Initial Alert - Inject - Control Room Operator to On Scene Commander - \"H2S gas detectors in Unit OD1 have triggered an alarm indicating elevated H2S levels.\" - On Scene Commander initiates initial assessment and alerts Emergency Response Team (ERT). - Tests early detection and initial notification procedures under OPITO MEMIR. - Ensure rapid recognition and communication of hazardous gas detection.\n\n2. 00:05 - Situation Confirmation - Inject - On Scene Commander to HSE Officer and ERT Leader - \"Confirm H2S concentration levels and assess affected areas.\" - HSE Officer verifies gas levels; ERT Leader prepares personnel with breathing apparatus. - Tests hazard assessment and PPE deployment readiness.\n\n3. 00:10 - Emergency Declaration - Inject - On Scene Commander to Company HQ Observer and all platform personnel - \"Declare H2S gas release emergency; initiate emergency shutdown procedures and evacuation protocols.\" - Control Room Operator initiates emergency shutdown; Deck Supervisor coordinates personnel evacuation to muster points. - Tests emergency declaration, shutdown, and evacuation coordination.\n\n4. 00:15 - External Notification - Inject - On Scene Commander to Coast Guard and Environmental Regulator - \"Notify external agencies of H2S release and potential environmental impact.\" - Coast Guard prepares for possible offshore assistance; Environmental Regulator requests monitoring data. - Tests external communication and regulatory notification.\n\n5. 00:20 - Environmental Concern Raised - Inject - Fisherman Representative to On Scene Commander - \"Concern expressed about potential impact on nearby fishing zones.\" - On Scene Commander acknowledges and coordinates with Environmental Regulator for impact assessment. - Tests stakeholder communication and environmental risk management.\n\n6. 00:25 - Medical Alert - Inject - Medical Officer to On Scene Commander - \"Report of personnel exhibiting symptoms consistent with H2S exposure.\" - Medical Officer initiates medical triage and requests emergency medical evacuation if needed. - Tests medical response and casualty management.\n\n7. 00:30 - Spill Containment - Inject - ERT Leader to Logistics Coordinator - \"Report of minor hydrocarbon oil spill near containment area.\" - Logistics Coordinator deploys spill kits and containment measures. - Tests spill response and resource coordination.\n\n8. 00:40 - Communication System Check - Inject - Control Room Operator to all teams - \"Verify functionality of all communication systems including VHF radios and satellite phones.\" - Teams confirm operational status; report any failures. - Tests communication reliability under emergency conditions.\n\n9. 00:50 - Emergency Shutdown Verification - Inject - Control Room Operator to On Scene Commander - \"Confirm all emergency shutdown systems are fully engaged and stable.\" - On Scene Commander verifies system status and reports to Company HQ Observer. - Tests shutdown system effectiveness and reporting.\n\n10. 01:00 - Situation Update and Demobilization Planning - Inject - On Scene Commander to all response teams and Company HQ Observer - \"Provide status update; begin planning phased demobilization as gas levels return to safe limits.\" - Teams prepare for safe return to normal operations following OPITO MEMIR guidelines. - Tests incident resolution, recovery planning, and debriefing procedures.")

Based on the confirmed details, here is the Master Scenario Events List: 

1. 00:00 - Initial Alert - Inject - Control Room Operator to On Scene Commander - "H2S gas detectors in Unit OD1 have triggered an alarm indicating elevated H2S levels." - On Scene Commander initiates initial assessment and alerts Emergency Response Team (ERT). - Tests early detection and initial notification procedures under OPITO MEMIR. - Ensure rapid recognition and communication of hazardous gas detection.

2. 00:05 - Situation Confirmation - Inject - On Scene Commander to HSE Officer and ERT Leader - "Confirm H2S concentration levels and assess affected areas." - HSE Officer verifies gas levels; ERT Leader prepares personnel with breathing apparatus. - Tests hazard assessment and PPE deployment readiness.

3. 00:10 - Emergency Declaration - Inject - On Scene Commander to Company HQ Observer and all platform personnel - "Declare H2S gas release emergency; initiate emergency shutdown procedures and evacuation

In [11]:
def extract_required_data(data: dict) -> dict:
    """
    Extract required data from the input dictionary.
    """
    REQUIRED_FIELDS = {
        "unit_name": None,
        "unit_type": None,
        "asset_name": None,
        "asset_type": None,
        "asset_location": None,
        "ownership_operator_name": None,
        "workforce_size_shift": None,
        "primary_function": None,
        "key_processes": None,
        "hazardous_materials": None,
        "response_equipment": None,
        "communication_systems": None,
        "environmental_conditions": None,
        "proximity_sensitive_areas": None,
        "trainee_roles": None,
    }

    collected = data.get("collectedUnitInfo", {})
    print(collected)
    # Check if *all* values are None
    # all_null = all(v is None for v in collected.values())

    if collected != None:
        # Use collectedUnitInfo if available
        REQUIRED_FIELDS.update({
            "unit_name": collected.get("unitName"),
            "unit_type": collected.get("unitType"),
            "asset_name": collected.get("assetName"),
            "asset_type": collected.get("assetType"),
            "asset_location": collected.get("assetLocation"),
            "ownership_operator_name": collected.get("ownershipOperatorName"),
            "workforce_size_shift": collected.get("workforceSizeShift"),
            "primary_function": collected.get("primaryFunction"),
            "key_processes": collected.get("keyProcesses"),
            "hazardous_materials": collected.get("hazardousMaterials"),
            "response_equipment": collected.get("responseEquipment"),
            "communication_systems": collected.get("communicationSystems"),
            "environmental_conditions": collected.get("environmentalConditions"),
            "proximity_sensitive_areas": collected.get("proximitySensitiveAreas"),
        })
    else:
        # Fallback to unitInfo
        unit_info = data.get("unitInfo", {})
        asset = unit_info.get("asset", {})
        org = asset.get("organisation", {})

        REQUIRED_FIELDS.update({
            "unit_name": unit_info.get("name"),
            "unit_type": unit_info.get("type"),
            "asset_name": asset.get("name"),
            "asset_type": asset.get("type"),
            "asset_location": asset.get("country"),
            "ownership_operator_name": org.get("name"),
        })

    # Extract trainee roles from teamsAssigned
    teams = data.get("teamsAssigned", [])
    trainee_roles = []
    for team in teams:
        for user in team.get("users", []):
            role = user.get("roleName")
            if role:
                trainee_roles.append(role)

    REQUIRED_FIELDS["trainee_roles"] = ", ".join(trainee_roles) if trainee_roles else None

    # Return as JSON string
    return json.dumps(REQUIRED_FIELDS, indent=2)

In [3]:
data = {
  "unitInfo": {
    "name": "OD1",
    "type": "crude oil distillation unit",
    "asset": {
      "name": "OffShore 1",
      "type": "offshore production platform",
      "coordinates": "coordinates",
      "country": "Saudi Arabia",
      "organisation": {
        "name": "saudi LNG",
        "description": "a global organisation focused on oil and natural gas production",
        "hqLocation": "Saudi Arabia",
        "country": "Saudi arabia"
      }
    }
  },
  "teamsAssigned": [
    {
      "name": "emergency response 1",
      "users": [
        {
          "roleName": "first aid responder"
        }
      ]
    },
    {
      "name": "second team",
      "users": []
    }
  ],
  "collectedUnitInfo": {
    "unitName": "Test Unit",
    "unitType": "Test Unit Type",
    "assetName": "City of Port of Spain",
    "assetType": "Coastal Region",
    "assetLocation": "Port of Spain, Trinidad & Tobago",
    "ownershipOperatorName": "Galileo Org",
    "workforceSizeShift": "150 personnel, 3 shifts per day",
    "primaryFunction": None,
    "keyProcesses": None,
    "hazardousMaterials": None,
    "responseEquipment": None,
    "communicationSystems": None,
    "environmentalConditions": None,
    "proximitySensitiveAreas": None
  }
}

In [12]:
data2 = {
  "unitInfo": {
    "name": "OD1",
    "type": "crude oil distillation unit",
    "asset": {
      "name": "OffShore 1",
      "type": "offshore production platform",
      "coordinates": "coordinates",
      "country": "Saudi Arabia",
      "organisation": {
        "name": "saudi LNG",
        "description": "a global organisation focused on oil and natural gas production",
        "hqLocation": "Saudi Arabia",
        "country": "Saudi arabia"
      }
    }
  },
  "teamsAssigned": [
    {
      "name": "emergency response 1",
      "users": [
        {
          "roleName": "first aid responder"
        },
        {
          "roleName": "safety officer"
        }
      ]
    },
    {
      "name": "second team",
      "users": [
          {
              "roleName": "firefighter"
          }
      ]
    }
  ],
  "collectedUnitInfo": { }
}

In [ ]:
data3 = {
    "unitInfo": {
        "name": "OD1",
        "type": "crude oil distillation unit",
        "asset": {
            "name": "OffShore 1",
            "type": "offshore production platform",
            "coordinates": "coordinates",
            "country": "Saudi Arabia",
            "organisation": {
                "name": "saudi LNG",
                "description": "a global organisation focused on oil and natural gas production.",
                "hqLocation": "Saudi Arabia",
                "country": "Saudi Arabia"
            }
        }
    },
    "teamsAssigned": [
        {
            "name": "Critical Care Crew",
            "users": []
        }
    ],
    "collectedUnitInfo": null
}.json()

In [13]:
print(extract_required_data(data2))

{
  "unit_name": "OD1",
  "unit_type": "crude oil distillation unit",
  "asset_name": "OffShore 1",
  "asset_type": "offshore production platform",
  "asset_location": "Saudi Arabia",
  "ownership_operator_name": "saudi LNG",
  "workforce_size_shift": null,
  "primary_function": null,
  "key_processes": null,
  "hazardous_materials": null,
  "response_equipment": null,
  "communication_systems": null,
  "environmental_conditions": null,
  "proximity_sensitive_areas": null,
  "trainee_roles": "first aid responder, safety officer, firefighter"
}


In [9]:
type(extract_required_data(data2))

str

In [8]:
UNIT_API_BASE = "http://134.209.148.133:3001/api/ai-scenario-generation/hierarchy/unit"

In [9]:
resp = requests.get(f"{UNIT_API_BASE}/{'7'}")
print(resp.json())
print(type(resp.json()))

{'unitInfo': {'name': 'OD1', 'type': 'crude oil distillation unit', 'asset': {'name': 'OffShore 1', 'type': 'offshore production platform', 'coordinates': 'coordinates', 'country': 'Saudi Arabia', 'organisation': {'name': 'saudi LNG', 'description': 'a global organisation focused on oil and natural gas production.', 'hqLocation': 'Saudi Arabia', 'country': 'Saudi Arabia'}}}, 'teamsAssigned': [{'name': 'Critical Care Crew', 'users': []}], 'collectedUnitInfo': None}
<class 'dict'>


In [12]:
print(extract_required_data(resp.json()))

None
{
  "unit_name": "OD1",
  "unit_type": "crude oil distillation unit",
  "asset_name": "OffShore 1",
  "asset_type": "offshore production platform",
  "asset_location": "Saudi Arabia",
  "ownership_operator_name": "saudi LNG",
  "workforce_size_shift": null,
  "primary_function": null,
  "key_processes": null,
  "hazardous_materials": null,
  "response_equipment": null,
  "communication_systems": null,
  "environmental_conditions": null,
  "proximity_sensitive_areas": null,
  "trainee_roles": null
}
